In [8]:
!pip install albumentations

/bin/bash: /home/bryan_santosa/miniconda3/envs/machinelearning/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached simsimd-6.5.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (70 kB)
Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached simsimd-6.5.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (1.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.2/592.2 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [albumentations]m [albumentations]


In [13]:
import numpy as np
import torch # If you use PyTorch later for training
import random
import os

# Choose an integer seed (e.g., 42, 0, or any number)
SEED = 42

def set_all_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # If you use PyTorch, uncomment these:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds(SEED)

In [14]:
import cv2
import albumentations as A
from glob import glob

# Paths
INPUT_DATA_ROOT = "../data/raw/Indonesian License Plate Dataset/"

input_images = os.path.join(INPUT_DATA_ROOT, "images")
input_labels = os.path.join(INPUT_DATA_ROOT, "labels")

# OUTPUT Paths (A new folder to store the augmented data, usually outside of 'raw')
OUTPUT_DATA_ROOT = "../data/processed/"

output_images = os.path.join(OUTPUT_DATA_ROOT, "images")
output_labels = os.path.join(OUTPUT_DATA_ROOT, "labels")

# --- END: Updated Paths ---

# Create output directories
os.makedirs(output_images, exist_ok=True)
os.makedirs(output_labels, exist_ok=True)

# Define augmentation pipeline
transform = A.Compose([
    A.RandomRotate90(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.MotionBlur(blur_limit=5, p=0.3),
    A.RandomScale(scale_limit=0.2, p=0.5),
],
bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'],
    # Add clip=True to ensure all coordinates are strictly within [0.0, 1.0]
    clip=True,
    min_area=1.0
))

# Loop through all images
for subdir in os.listdir(input_images):
    # Construct the full paths for the current subdirectory
    current_input_image_dir = os.path.join(input_images, subdir)
    current_input_label_dir = os.path.join(input_labels, subdir)

    # 🛑 FIX 2: Create the corresponding output subdirectories
    current_output_image_dir = os.path.join(output_images, subdir)
    current_output_label_dir = os.path.join(output_labels, subdir)

    os.makedirs(current_output_image_dir, exist_ok=True)
    os.makedirs(current_output_label_dir, exist_ok=True)
    
    print(f"Processing directory: {subdir}")

    # Loop through all images in the subdirectory
    # Using os.path.join is safer than f-string concatenation for paths
    for img_path in glob(os.path.join(current_input_image_dir, "*.jpg")):
        filename = os.path.basename(img_path)
        
        # 🛑 FIX 1: Correctly construct the label path using the subdirectory path
        label_path = os.path.join(current_input_label_dir, filename.replace(".jpg", ".txt"))

        # Safety Check: Ensure the label file exists before proceeding
        if not os.path.exists(label_path):
            print(f"Warning: Label file not found for {filename} in {subdir}. Skipping.")
            continue
            
        # Read image and labels (continue with your original logic)
        image = cv2.imread(img_path)
        if image is None:
            print(f"Error: Could not read image {img_path}. Skipping.")
            continue
            
        h, w, _ = image.shape # This is for context, not strictly needed by the augmentation code

        # ... (rest of the label reading, augmentation, and saving logic) ...

        with open(label_path, "r") as f:
            labels = f.readlines()

        bboxes = []
        class_labels = []
        for label in labels:
            try:
                cls, x, y, bw, bh = map(float, label.strip().split())
                bboxes.append([x, y, bw, bh])
                class_labels.append(int(cls))
            except Exception as e:
                 # Added error handling for malformed lines
                print(f"Error processing label line in {label_path}: {e}. Skipping image.")
                continue

        # Apply transformation
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)

        aug_img = transformed['image']
        aug_bboxes = transformed['bboxes']
        aug_labels = transformed['class_labels']

        # 🛑 FIX 2 (Final Save): Save augmented files into the correct OUTPUT subdirectory
        
        # Adding 'aug_' prefix is good practice to distinguish augmented files
        aug_filename = f"aug_{filename}" 

        # Save augmented image
        save_path_img = os.path.join(current_output_image_dir, aug_filename)
        cv2.imwrite(save_path_img, aug_img)

        # Save updated YOLO labels
        save_path_lbl = os.path.join(current_output_label_dir, aug_filename.replace(".jpg", ".txt"))
        with open(save_path_lbl, "w") as f:
            for cls, bbox in zip(aug_labels, aug_bboxes):
                f.write(f"{cls} {' '.join(map(str, bbox))}\n")

print("✅ Data augmentation complete!")


Processing directory: test
Processing directory: train
Processing directory: val
✅ Data augmentation complete!


In [3]:
from ultralytics import YOLO
import torch

print("YOLO version:", YOLO._version)
print("CUDA available:", torch.cuda.is_available())
print("Device being used:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


YOLO version: 1
CUDA available: True
Device being used: NVIDIA GeForce RTX 3060 Laptop GPU


In [4]:
import torch

# Returns the number of CUDA devices detected (e.g., 1, 2, 4)
print(torch.cuda.device_count())

1


In [5]:
import torch

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        # Prints the index and the GPU model name
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Check your PyTorch installation and NVIDIA drivers.")

GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU


In [6]:
import torch

# Get the name of the currently selected device
print("Current Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

# Check total and allocated memory on the current GPU (useful for debugging OOM errors)
print("Total VRAM:", torch.cuda.get_device_properties(0).total_memory / 1024**3, "GB")
print("Allocated VRAM:", torch.cuda.memory_allocated(0) / 1024**3, "GB")

Current Device Name: NVIDIA GeForce RTX 3060 Laptop GPU
Total VRAM: 5.99951171875 GB
Allocated VRAM: 0.0 GB


In [20]:
from ultralytics import YOLO

DATA_YAML_PATH = '../data.yaml'

target_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("CUDA available:", torch.cuda.is_available())
print("YOLO target device:", target_device)

# Load a pre-trained YOLOv8-nano model
model = YOLO('yolov8n.pt')

model.model.to(target_device) 

# 3. Verify the change (using the reliable check)
first_param_device = next(model.model.parameters()).device

print("✅ Model device (PyTorch check):", first_param_device)

print("Starting YOLOv8 training...")
results = model.train(
    data=DATA_YAML_PATH,     # Path to the data configuration file
    epochs=40,              # Number of epochs (adjust as needed)
    imgsz=640,               # Input image size (standard for YOLO)
    batch=6,                # Batch size (reduce if you run out of GPU memory)
    name='lp_detection_fine_tune_v2',  # Name for the results folder in 'runs/detect
    # Optional: use the 'freeze' argument if you want to freeze the backbone 
    amp=True,
    lr0=1e-3
)

print("Training complete. Results saved in runs/detect/lp_detection_v1")

CUDA available: True
YOLO target device: cuda
✅ Model device (PyTorch check): cuda:0
Starting YOLOv8 training...
Ultralytics 8.3.222 🚀 Python-3.10.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=lp_detection_fine_t

Traceback (most recent call last):
  File "/home/bryan_santosa/miniconda3/envs/machinelearning/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/home/bryan_santosa/miniconda3/envs/machinelearning/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/bryan_santosa/miniconda3/envs/machinelearning/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


       1/40      1.64G      1.554      2.802      1.157          4        640: 100% ━━━━━━━━━━━━ 134/134 4.9it/s 27.2s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 7.8it/s 1.2s0.1s
                   all        100        179      0.874      0.726      0.838      0.438

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/40      1.72G      1.495      1.758      1.136          6        640: 100% ━━━━━━━━━━━━ 134/134 7.7it/s 17.4s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 9.5it/s 0.9s0.1s
                   all        100        179      0.754      0.799      0.858      0.535

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/40      1.72G       1.48      1.602       1.13          5        640: 100% ━━━━━━━━━━━━ 134/134 7.4it/s 18.2s0.1ss
                 Class     Images  I

In [4]:
evaluate = model.val()

Ultralytics 8.3.222 🚀 Python-3.10.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 1.6±0.2 ms, read: 119.7±15.2 MB/s, size: 1961.0 KB)
val: Scanning /mnt/d/Uni/Semester 5/Computer Vision/Plate-Recognition/data/raw/Indonesian License Plate Dataset/labels/val.cache... 100 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 164.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.6it/s 4.3s0.3ss
                   all        100        179      0.972      0.954      0.988      0.655
Speed: 4.7ms preprocess, 6.9ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to /mnt/d/Uni/Semester 5/Computer Vision/Plate-Recognition/notebooks/runs/detect/val
